In [1]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 


In [2]:
import pandas as pd
import numpy as np

In [3]:
# Confirm folder was created and files added successfully
os.listdir("Data/")


['.ipynb_checkpoints',
 'basics.csv',
 'IMDB Movie Dataset Info.docx',
 'ratings.csv',
 'title-akas-us-only.csv',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz']

## Importing title.akas

In [4]:
akas = pd.read_csv('Data/title-akas-us-only.csv', low_memory=False)



In [5]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1452564 entries, 0 to 1452563
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1452564 non-null  object
 1   ordering         1452564 non-null  int64 
 2   title            1452564 non-null  object
 3   region           1452564 non-null  object
 4   language         1452564 non-null  object
 5   types            1452564 non-null  object
 6   attributes       1452564 non-null  object
 7   isOriginalTitle  1452564 non-null  object
dtypes: int64(1), object(7)
memory usage: 88.7+ MB


## Importing title.basics

In [6]:
basics = pd.read_csv('Data/title.basics.tsv.gz', sep='\t', low_memory=False)


In [7]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10017011 entries, 0 to 10017010
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 687.8+ MB


In [8]:
# Filter the basics table down to only include the US by using the filter akas dataframe
filter_us_titles = basics['tconst'].isin(akas['titleId'])
basics = basics[filter_us_titles]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"
...,...,...,...,...,...,...,...,...,...
10016872,tt9916560,tvMovie,March of Dimes Presents: Once Upon a Dime,March of Dimes Presents: Once Upon a Dime,0,1963,\N,58,Family
10016901,tt9916620,movie,The Copeland Case,The Copeland Case,0,\N,\N,\N,Drama
10016939,tt9916702,short,Loving London: The Playground,Loving London: The Playground,0,\N,\N,\N,"Drama,Short"
10016962,tt9916756,short,Pretty Pretty Black Girl,Pretty Pretty Black Girl,0,2019,\N,\N,Short


In [9]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
5,tt0000006,short,Chinese Opium Den,Chinese Opium Den,0,1894,\N,1,Short
6,tt0000007,short,Corbett and Courtney Before the Kinetograph,Corbett and Courtney Before the Kinetograph,0,1894,\N,1,"Short,Sport"


## Preprocessing steps for title basics

In [10]:
## changing \N to actual Nan values

In [11]:
basics = basics.replace({'\\N':np.nan})

In [12]:
## dropping Nan's from runtimeMinutes and genres columns

In [13]:
basics.dropna(subset=['runtimeMinutes', 'genres'], inplace=True)

In [14]:
## creating filters to keep only movie in titletype, and remove documentary from genres

In [15]:
filter_titletype = basics['titleType'].str.contains('movie')
filter_documentaries = basics['genres'].str.contains('Documentary')

In [16]:
basics = basics[~filter_documentaries]

In [17]:
basics = basics[filter_titletype]

C:\Users\Admin\AppData\Local\Temp\ipykernel_59068\2716687499.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  basics = basics[filter_titletype]


In [18]:
# verifying changes

In [19]:
basics['genres'].value_counts()

Drama                        30948
Comedy                       13170
Comedy,Drama                  6332
Horror                        5535
Drama,Romance                 5479
                             ...  
Action,Mystery,War               1
Animation,Musical,Romance        1
Drama,News,Thriller              1
Fantasy,Music,Mystery            1
Biography,Fantasy,Musical        1
Name: genres, Length: 1058, dtype: int64

In [20]:
basics['titleType'].value_counts()

movie    162615
Name: titleType, dtype: int64

In [21]:
# change startYear dtype to float

In [22]:
basics['startYear'] = basics['startYear'].astype(float)

In [23]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 162615 entries, 8 to 10016777
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          162615 non-null  object 
 1   titleType       162615 non-null  object 
 2   primaryTitle    162615 non-null  object 
 3   originalTitle   162615 non-null  object 
 4   isAdult         162615 non-null  object 
 5   startYear       159998 non-null  float64
 6   endYear         0 non-null       object 
 7   runtimeMinutes  162615 non-null  object 
 8   genres          162615 non-null  object 
dtypes: float64(1), object(8)
memory usage: 12.4+ MB


In [24]:
# filter to only keep movies between startyear 2000 and 2022

In [25]:
filter_startyear = (basics['startYear'] >=2000.0) & (basics['startYear'] <=2022.0)

In [26]:
basics = basics[filter_startyear]

In [27]:
## final info check to view rows count

In [28]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86979 entries, 34802 to 10016777
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          86979 non-null  object 
 1   titleType       86979 non-null  object 
 2   primaryTitle    86979 non-null  object 
 3   originalTitle   86979 non-null  object 
 4   isAdult         86979 non-null  object 
 5   startYear       86979 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  86979 non-null  object 
 8   genres          86979 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.6+ MB


In [29]:
## export dataframe to csv in data folder

In [30]:
basics.to_csv(r'Data\basics.csv')

## Load and filter the title ratings file

In [31]:
ratings = pd.read_csv('Data/title.ratings.tsv.gz', sep='\t', low_memory=False)

In [32]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1331492 entries, 0 to 1331491
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1331492 non-null  object 
 1   averageRating  1331492 non-null  float64
 2   numVotes       1331492 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 30.5+ MB


In [33]:
filter_basics = ratings['tconst'].isin(basics['tconst'])
ratings = ratings[filter_basics]


In [34]:
ratings = ratings.replace({'\\N':np.nan})

In [35]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71900 entries, 17961 to 1331462
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         71900 non-null  object 
 1   averageRating  71900 non-null  float64
 2   numVotes       71900 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 2.2+ MB


In [36]:
ratings.to_csv(r'Data\ratings.csv')

## Part 2

In [37]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34802,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61114,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
67666,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86793,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93930,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [38]:
ratings.head()

,tconst,averageRating,numVotes
17961,tt0035423,6.4,87153
40764,tt0062336,6.4,175
46645,tt0069049,6.7,7754
63640,tt0088751,5.2,336
69953,tt0096056,5.6,846


In [54]:
basics.dtypes

tconst             object
titleType          object
primaryTitle       object
originalTitle      object
isAdult            object
startYear         float64
endYear            object
runtimeMinutes     object
genres             object
dtype: object

In [55]:
ratings.dtypes

tconst            object
averageRating    float64
numVotes           int64
dtype: object

In [57]:
basics['tconst'].map(len).max()

10

In [58]:
basics['primaryTitle'].map(len).max()

242

In [60]:
basics['genres'].map(len).max()

29